In [41]:
import re
import os
from openai import AzureOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
from datetime import datetime
import pandas as pd
import docx2txt
import json
from tqdm import tqdm
from pathlib import Path as p
import urllib
import warnings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
import pandas as pd
import asyncio
import uuid
import tiktoken
from gensim.parsing.preprocessing import remove_stopwords
import operator
from typing import Annotated, List, Literal, TypedDict

from langchain.chains.combine_documents.reduce import (
    acollapse_docs,
    split_list_of_docs,
)
from langchain_core.documents import Document
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph


warnings.filterwarnings("ignore")

In [42]:
encoding = tiktoken.get_encoding("cl100k_base")
os.environ["AZURE_OPENAI_API_KEY"] = "ee65aca022a74803b2e2d1ff4c373b05"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://firstsource.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt4"

In [55]:
parameters_prompt = '''You are a sales assistant tasked with summarizing email or phone conversations between a university salesperson and a prospective student. Based on the provided email summary, your goal is to construct a comprehensive student profile, develop a targeted campaign, craft a personalized sales pitch, and assess the student’s interest level in the university's program.

Student Persona
Name: (Student’s name if available, otherwise "NA")
Age: (Age if available otherwise mark as "NA")
Location: (Student's location, or mark as "NA")
Interest: (Summarize the student’s interest in the program or field of study. If not stated, mark as "NA")
Behavior: (Describe any behavioral traits or actions from the emails that indicate the student's approach to education, such as responsiveness, motivation, or any other relevant insights)
Dislikes: (Mention any dislikes expressed in the conversation about education, programs, or formats. Mark as "NA" if unavailable)
Likes: (Include any expressed likes, such as flexibility, subject matter, or specific university features. Mark as "NA" if not mentioned)
Modes of Communication: (Identify all modes of communication between Salesperson and Student)
Preferred Communication: (Identify the student's preferred communication method—email, phone, etc. Mark as "NA" if not specified)

Potential Campaign
Theme: (Craft a campaign theme based on the student’s interests and persona. Example: "Empowering Working Professionals with Flexible Learning")
Content Focus: (Outline the content to feature in this campaign, e.g., "Showcase alumni success stories, emphasize career advancement opportunities, highlight flexible schedules")
Platforms: (List suitable platforms for the campaign, such as "Email marketing, webinars, blog posts, or social media ads")

Sales Strategy
Personalized Sales Pitch: (Write a customized message addressing the student's needs, aspirations, and any concerns.

Sentiment
Student Sentiment: (Assess the overall sentiment from the conversation, such as "Excited", "Curious", "Optimistic", "Anxious", "Confused", "Skeptical", "Indecisive", "Disappointed", "Confident", "Motivated", "Overwhelmed", "Frustrated", "Reluctant", "Satisfied" into a single sentiment.)

Lead Score
Lead Interest Level: (Rate the student’s potential interest in the program: Low, Medium, or High, based on the tone and content of the conversation)'''

In [44]:
llm = AzureChatOpenAI(
api_key=os.environ["AZURE_OPENAI_API_KEY"],
api_version=os.environ["AZURE_OPENAI_API_VERSION"],
azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
model='gpt-4o'
)

In [45]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [46]:
token_max = 1000


def length_function(documents: List[Document]) -> int:
    """Get number of tokens for input contents."""
    return sum(llm.get_num_tokens(doc.page_content) for doc in documents)


class OverallState(TypedDict):
    contents: List[str]
    summaries: Annotated[list, operator.add]
    collapsed_summaries: List[Document]
    final_summary: str

class SummaryState(TypedDict):
    content: str


async def generate_summary(state: SummaryState):
    response = await map_chain.ainvoke(state["content"])
    return {"summaries": [response]}


def map_summaries(state: OverallState):
    return [
        Send("generate_summary", {"content": content}) for content in state["contents"]
    ]


def collect_summaries(state: OverallState):
    return {
        "collapsed_summaries": [Document(summary) for summary in state["summaries"]]
    }


async def collapse_summaries(state: OverallState):
    doc_lists = split_list_of_docs(
        state["collapsed_summaries"], length_function, token_max
    )
    results = []
    for doc_list in doc_lists:
        results.append(await acollapse_docs(doc_list, reduce_chain.ainvoke))

    return {"collapsed_summaries": results}


def should_collapse(
    state: OverallState,
) -> Literal["collapse_summaries", "generate_final_summary"]:
    num_tokens = length_function(state["collapsed_summaries"])
    if num_tokens > token_max:
        return "collapse_summaries"
    else:
        return "generate_final_summary"


async def generate_final_summary(state: OverallState):
    response = await reduce_chain.ainvoke(state["collapsed_summaries"])
    return {"final_summary": response}


graph = StateGraph(OverallState)
graph.add_node("generate_summary", generate_summary)  # same as before
graph.add_node("collect_summaries", collect_summaries)
graph.add_node("collapse_summaries", collapse_summaries)
graph.add_node("generate_final_summary", generate_final_summary)

graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
graph.add_edge("generate_summary", "collect_summaries")
graph.add_conditional_edges("collect_summaries", should_collapse)
graph.add_conditional_edges("collapse_summaries", should_collapse)
graph.add_edge("generate_final_summary", END)

app = graph.compile()

In [47]:
map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise one-liner summary of the following conversation, including any student information mentioned, the date of the conversation, the communication channel (phone, email, WhatsApp), the student's level of interest (Low, Medium, or High), and their emotional state (Excited, Curious, Optimistic, Anxious, Confused, Skeptical, Indecisive, Disappointed, Confident, Motivated, Overwhelmed, Frustrated, Neutral, Reluctant, or Satisfied):\n\n{context}")])

map_chain = map_prompt | llm | StrOutputParser()

In [48]:
reduce_template = """
The following are summaries of conversations in no particular order:
{docs}
Please organize these summaries in ascending order by date if available. If dates are not provided, maintain their original order.

"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

reduce_chain = reduce_prompt | llm | StrOutputParser()

In [49]:
docs = [Document(page_content=('''--- Content of 10_Apr_2024_0338_Adeola_Oyetunji.docx ---

Subject:Re: UCLan Online - Application Transfer

10 Apr 2024 at 03:38

I am still struggling to apply for my student finance but I will try and sort it after my admission has been confirmed. 

Thank you 
Adeola 

On Monday, 8 April 2024 at 09:17:21 BST, Henry Norman <> wrote:


Hey Adeola,

Brilliant, thank you for getting this sorted. 

I will get the admissions team to transfer you over to the online programme ASAP. 
We should hopefully issue you a response very soon. 

Have you been able to apply for your student finance at all?

Let me know if you have any questions.

Best Wishes,
Henry Norman
UCLan ONLINE | 
Email: 
Tel:  | WhatsApp: 
LinkedIn: 





From: Adeola Oyetunji <>
Sent: Saturday, April 6, 2024 5:31 AM
To: Henry Norman (Staff) <>
Subject: Re: UCLan Online - Application Transfer
 
Hi Henry,

I wish to apply for MSc Nursing, part time to start in May. Thank you 
Adeola 

On Thursday, 4 April 2024 at 13:11:15 BST, Henry Norman <> wrote:


Hey Adeola,

We have found your application and we need to have it transferred to Online and for the MSc Nursing programme.

Please can you email a response to me and confirm that you wish:
To Study MSc Nursing
To Study Online and Part-time
To Start in the May intake
I very much look forward to your response, as soon as we can get this transferred the sooner we can get you a response from the admissions team!

Best Wishes,
Henry Norman
UCLan ONLINE | 
Email: 
Tel:  | WhatsApp: 
LinkedIn: 






--- Content of 10_Apr_2024_1514_Adeola_Oyetunji.docx ---

Subject:Re: UCLan Online - Application Transfer

10 Apr 2024 at 15:14 GMT

Hey Adeola,

What are issues are you facing when trying to apply for SFE? The process should be quite simple and just follow the prompts when you apply.

Let me know.

Best Wishes,
Henry Norman
UCLan ONLINE | 
Email: 
Tel:  | WhatsApp: 
LinkedIn: 





I am still struggling to apply for my student finance but I will try and sort it after my admission has been confirmed. 

Thank you 
Adeola 

On Monday, 8 April 2024 at 09:17:21 BST, Henry Norman <> wrote:


Hey Adeola,

Brilliant, thank you for getting this sorted. 

I will get the admissions team to transfer you over to the online programme ASAP. 
We should hopefully issue you a response very soon. 

Have you been able to apply for your student finance at all?

Let me know if you have any questions.

Best Wishes,
Henry Norman
UCLan ONLINE | 
Email: 
Tel:  | WhatsApp: 
LinkedIn: 





From: Adeola Oyetunji <>
Sent: Saturday, April 6, 2024 5:31 AM
To: Henry Norman (Staff) <>
Subject: Re: UCLan Online - Application Transfer
 
Hi Henry,

I wish to apply for MSc Nursing, part time to start in May. Thank you 
Adeola 

On Thursday, 4 April 2024 at 13:11:15 BST, Henry Norman <> wrote:


Hey Adeola,

We have found your application and we need to have it transferred to Online and for the MSc Nursing programme.

Please can you email a response to me and confirm that you wish:
To Study MSc Nursing
To Study Online and Part-time
To Start in the May intake
I very much look forward to your response, as soon as we can get this transferred the sooner we can get you a response from the admissions team!

Best Wishes,
Henry Norman
UCLan ONLINE | 
Email: 
Tel:  | WhatsApp: 
LinkedIn: 
'''), metadata={"source":"local"})]

In [50]:
# docs

In [51]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

async for step in app.astream(
    {"contents": [doc.page_content for doc in split_docs]},
    {"recursion_limit": 10},
):
    print(list(step.keys()))
    
final_summary = step['generate_final_summary']['final_summary']

Generated 1 documents.
['generate_summary']
['collect_summaries']
['generate_final_summary']


In [32]:
import os
import requests

# Configuration
API_KEY = "3705201e84d74f23b1153f93a6b2defd"
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": parameters_prompt
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": final_summary
        }
      ]
    }
  ],
  "temperature": 0.1,
  "top_p": 0.95,
  "max_tokens": 800
}

ENDPOINT = "https://finetuningtest3607433214.openai.azure.com/openai/deployments/gpt-4o-2024-08-06-studentdata/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Hello! How can I assist you today?', 'role': 'assistant'}}], 'created': 1727035227, 'id': 'chatcmpl-AAN2payX5W7T43aNVyCLnbdX8AJc9', 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'system_fin

In [13]:
# final_response = response.json()

In [14]:
# print(parameters_prompt)

In [15]:
# print(final_response['choices'][0]['message']['content'])

In [16]:
# Convert the following code to a single function which takes 'data' as input

# docs = [Document(page_content=(data), metadata={"source":"local"})]
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=1000, chunk_overlap=0
# )
# split_docs = text_splitter.split_documents(docs)
# print(f"Generated {len(split_docs)} documents.")

# async for step in app.astream(
#     {"contents": [doc.page_content for doc in split_docs]},
#     {"recursion_limit": 10},
# ):
#     print(list(step.keys()))
    
# final_summary = step['generate_final_summary']['final_summary']

# payload = {
#   "messages": [
#     {
#       "role": "system",
#       "content": [
#         {
#           "type": "text",
#           "text": parameters_prompt
#         }
#       ]
#     },
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": final_summary
#         }
#       ]
#     }
#   ],
#   "temperature": 0.1,
#   "top_p": 0.95,
#   "max_tokens": 800
# }

# try:
#     response = requests.post(ENDPOINT, headers=headers, json=payload)
#     response.raise_for_status()
# except requests.RequestException as e:
#     raise SystemExit(f"Failed to make the request. Error: {e}")

# print(response.json())

In [17]:
async def process_data(data):
    # Create document
    docs = [Document(page_content=(data), metadata={"source": "local"})]
    
    # Initialize text splitter
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    
    # Split document into smaller chunks
    split_docs = text_splitter.split_documents(docs)
    print(f"Generated {len(split_docs)} documents.")
    
    # Generate final summary asynchronously
    async for step in app.astream(
        {"contents": [doc.page_content for doc in split_docs]},
        {"recursion_limit": 10},
    ):
        print(list(step.keys()))
    
    final_summary = step['generate_final_summary']['final_summary']
    
    # Prepare payload for API request
    payload = {
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": parameters_prompt
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": final_summary
            }
          ]
        }
      ],
      "temperature": 0.1,
      "top_p": 0.95,
      "max_tokens": 800
    }
    
    # Make API request and handle errors
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")
    
    # Print and return the API response
#     print(response.json())
    return response.json()

In [29]:
import requests

In [33]:
async def main():
    # Replace with actual parameters
    text = input()
    num_tokens = num_tokens_from_string(text, "cl100k_base")
    if num_tokens > 8000:
        data = remove_stopwords(text)
#         word_tokens = word_tokenize(data)
#         data = ' '.join([t for t in word_tokens if t not in stopwords.words('english')])
    else:
        data = text
    result = await process_data(data)
    return result['choices'][0]['message']['content']

In [34]:
hi = await main()

 --- Content of 10_Apr_2024_0338_Adeola_Oyetunji.docx ---  Subject:Re: UCLan Online - Application Transfer  10 Apr 2024 at 03:38  I am still struggling to apply for my student finance but I will try and sort it after my admission has been confirmed.   Thank you  Adeola   On Monday, 8 April 2024 at 09:17:21 BST, Henry Norman <> wrote:   Hey Adeola,  Brilliant, thank you for getting this sorted.   I will get the admissions team to transfer you over to the online programme ASAP.  We should hopefully issue you a response very soon.   Have you been able to apply for your student finance at all?  Let me know if you have any questions.  Best Wishes, Henry Norman UCLan ONLINE |  Email:  Tel:  | WhatsApp:  LinkedIn:       From: Adeola Oyetunji <> Sent: Saturday, April 6, 2024 5:31 AM To: Henry Norman (Staff) <> Subject: Re: UCLan Online - Application Transfer   Hi Henry,  I wish to apply for MSc Nursing, part time to start in May. Thank you  Adeola   On Thursday, 4 April 2024 at 13:11:15 BST, H

Generated 1 documents.
['generate_summary']
['collect_summaries']
['generate_final_summary']


In [35]:
print(hi)

Student Persona
Name: Adeola Oyetunji
Age: NA
Location: NA
Interest: Online MSc Nursing
Behavior: Responsive, motivated, seeking assistance with student finance application
Dislikes: NA
Likes: Flexibility of online program
Modes of Communication: Email
Preferred Communication: Email

Potential Campaign
Theme: "Supporting Your Journey to Online Nursing Excellence"
Content Focus: Showcase financial aid options, highlight flexible schedules, provide resources for a smooth transition to online learning
Platforms: Email marketing, webinars, blog posts, social media ads

Sales Strategy
Personalized Sales Pitch: Dear Adeola, we understand that transitioning to our Online MSc Nursing program is an exciting step in your career. We're here to support you every step of the way, especially with your student finance application. Our dedicated team will guide you through the process, ensuring a smooth transition. With our flexible online schedule, you'll be able to balance your studies with your per

### Langchain

In [5]:
os.environ["AZURE_OPENAI_API_KEY"] = "ee65aca022a74803b2e2d1ff4c373b05"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://firstsource.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt4"
ENDPOINT = "https://finetuningtest3607433214.openai.azure.com/openai/deployments/gpt-4o-2024-08-06-studentdata/chat/completions?api-version=2024-02-15-preview"

# llm = AzureChatOpenAI(
# api_key=os.environ["AZURE_OPENAI_API_KEY"],
# api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
# azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
# model='gpt-4o'
# )

In [52]:
import os

os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://finetuningtest3607433214.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "3705201e84d74f23b1153f93a6b2defd"

In [14]:
# Import Azure OpenAI
from langchain_openai import AzureOpenAI

In [30]:
# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(deployment_name="gpt-4o-2024-08-06-studentdata")

In [53]:
llm = AzureChatOpenAI(
api_key=os.environ["AZURE_OPENAI_API_KEY"],
api_version=os.environ["AZURE_OPENAI_API_VERSION"],
azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
azure_deployment="gpt-4o-2024-08-06-studentdata"
)

In [54]:
llm.invoke(final_summary).content

"Subject: Re: Application Process and Financial Concerns\n\nHi Adeola,\n\nI hope this message finds you well. I understand from your recent email that you're feeling frustrated with the financial application process. It's completely normal to feel this way, and I want to assure you that we're here to support you.\n\nRegarding your concern about sorting out finances after receiving your admission confirmation, it's a common approach for many students. We'll provide guidance and resources to help you navigate this process smoothly.\n\nAdditionally, I noticed you mentioned the possibility of transferring your application to the MSc Nursing program. If you'd like to explore this option further, please let us know, and we'll be happy to assist you with the necessary steps.\n\nRemember, you're not alone in this journey, and we're here to help you every step of the way.\n\nBest regards,\n\n[Your Name]  \n[Your Position]  \n[Your Contact Information]"

In [61]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", parameters_prompt),
    ("user", "{topic}")
])

prompt_template.invoke({"topic": final_summary}).messages[1].content

'1. 10 Apr 2024, Email, Adeola Oyetunji, Medium interest, Frustrated: Student struggles with applying for finance, will sort after admission confirmation; application transfer for MSc Nursing discussed.'

In [76]:
def add(a):
    return "Hi"

with gr.Blocks() as demo:
    with gr.Column(scale=1):
        text1 = gr.Textbox()
    with gr.Row():
        text2 = gr.Textbox()
    btn = gr.Button("Run")
    btn.click(fn=add, inputs=text1, outputs=text2)

demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [68]:
iface = gr.Interface(fn=add, inputs="text", outputs="text", title="Text Processing App")

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [63]:
import gradio as gr